In [1]:
!pwd

/home/nayeem/Desktop/MTZ/deepOffTarget/RNN Based Experiments


In [2]:
# root_path = '/home/user/Desktop/MTZ/CRISPR-RNN/'
root_path = '/home/nayeem/Desktop/MTZ/deepOffTarget/RNN Based Experiments/'
data_dir = root_path
data_path = data_dir + 'all_off_target.csv'
resource_dir = data_dir + "Resources/"

In [3]:
import os
import random
import torch
import numpy as np
import copy

seed = 12345

os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed_all(seed) 

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

/home/nayeem/anaconda3/envs/sa-test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
cd $resource_dir

/home/nayeem/Desktop/MTZ/deepOffTarget/RNN Based Experiments/Resources


#Models

In [6]:
import torch.nn as nn

class LSTM_Model_Generic(nn.Module):
    def __init__(self, config):
        super(LSTM_Model_Generic,self).__init__()
        # emb_size=256, hidden_size=128, hidden_layers=3, output=2

        self.vocab_size = config["vocab_size"]
        self.emb_size = config["emb_size"]
        self.hidden_size = config["hidden_size"]
        self.lstm_layers = config["lstm_layers"]
        self.bi_lstm = config["bi_lstm"]
        self.reshape = config["reshape"]

        self.number_hidden_layers = config["number_hidder_layers"]
        self.dropout_prob = config["dropout_prob"] 
        self.hidden_layers = []
        
        self.hidden_shape = self.hidden_size*2 if self.bi_lstm else self.hidden_size
        
        self.embedding = None
        if self.vocab_size > 0:
            self.embedding = nn.Embedding(self.vocab_size, self.emb_size, padding_idx=0)
        
        self.lstm= nn.LSTM(self.emb_size, self.hidden_size, num_layers=self.lstm_layers, 
                            batch_first=True, bidirectional=self.bi_lstm)
#         self.lstm= nn.GRU(self.emb_size, self.hidden_size, num_layers=self.lstm_layers, 
#                             batch_first=True, bidirectional=self.bi_lstm)

        start_size = self.hidden_shape

        self.relu = nn.ReLU
        # self.dropout = nn.Dropout(self.dropout_prob)

        for i in range(self.number_hidden_layers):
            self.hidden_layers.append(nn.Sequential(
                nn.Linear(start_size, start_size // 2),
                nn.ReLU(),
                nn.Dropout(self.dropout_prob)))
            
            start_size = start_size // 2
            
        self.hidden_layers = nn.ModuleList(self.hidden_layers)
        self.output = nn.Linear(start_size,2)
        
        
    def forward(self,x):
        dir = 2 if self.bi_lstm else 1
        h = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)
        c = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)
        
        if self.embedding is not None:
            x = x.type(torch.LongTensor).to(device)
            x = self.embedding(x)
        elif self.reshape:
            x = x.view(x.shape[0],x.shape[1],1)

        x, (hidden, cell) = self.lstm(x, (h,c))
        
        x = x[:, -1, :]

        # print(x.shape)
        for i, layer in enumerate(self.hidden_layers):
            x = layer(x)
            # print(x.shape)
        x = self.output(x)
        # print(x.shape)
        return x

In [7]:
import torch.nn as nn

class RNN_Model_Generic(nn.Module):
    def __init__(self, config, model_type):
        super(RNN_Model_Generic,self).__init__()
        # emb_size=256, hidden_size=128, hidden_layers=3, output=2

        self.model_type = model_type
        self.vocab_size = config["vocab_size"]
        self.emb_size = config["emb_size"]
        self.hidden_size = config["hidden_size"]
        self.lstm_layers = config["lstm_layers"]
        self.bi_lstm = config["bi_lstm"]
        self.reshape = config["reshape"]

        self.number_hidden_layers = config["number_hidder_layers"]
        self.dropout_prob = config["dropout_prob"] 
        self.hidden_layers = []
        
        self.hidden_shape = self.hidden_size*2 if self.bi_lstm else self.hidden_size
        
        self.embedding = None
        if self.vocab_size > 0:
            self.embedding = nn.Embedding(self.vocab_size, self.emb_size, padding_idx=0)
        
        
        if model_type == "LSTM":
            self.lstm = nn.LSTM(self.emb_size, self.hidden_size, num_layers=self.lstm_layers, 
                            batch_first=True, bidirectional=self.bi_lstm)
        elif model_type == "GRU":    
            self.lstm= nn.GRU(self.emb_size, self.hidden_size, num_layers=self.lstm_layers, 
                           batch_first=True, bidirectional=self.bi_lstm)
        else:
            self.lstm= nn.RNN(self.emb_size, self.hidden_size, num_layers=self.lstm_layers, 
                           batch_first=True, bidirectional=self.bi_lstm)

        start_size = self.hidden_shape

        self.relu = nn.ReLU
        # self.dropout = nn.Dropout(self.dropout_prob)

        for i in range(self.number_hidden_layers):
            self.hidden_layers.append(nn.Sequential(
                nn.Linear(start_size, start_size // 2),
                nn.ReLU(),
                nn.Dropout(self.dropout_prob)))
            
            start_size = start_size // 2
            
        self.hidden_layers = nn.ModuleList(self.hidden_layers)
        self.output = nn.Linear(start_size,2)
        
        
    def forward(self,x):
        dir = 2 if self.bi_lstm else 1
        h = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)
        c = torch.zeros((self.lstm_layers*dir, x.size(0), self.hidden_size)).to(device)
        
        if self.embedding is not None:
            x = x.type(torch.LongTensor).to(device)
            x = self.embedding(x)
        elif self.reshape:
            x = x.view(x.shape[0],x.shape[1],1)
        
        if self.model_type == "LSTM":
            x, (hidden, cell) = self.lstm(x, (h,c))
        else:     
            x, hidden = self.lstm(x, h)
        
        x = x[:, -1, :]

        # print(x.shape)
        for i, layer in enumerate(self.hidden_layers):
            x = layer(x)
            # print(x.shape)
        x = self.output(x)
        # print(x.shape)
        return x

#Training and Evaluation Loop

In [8]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class TrainerDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs= inputs
        self.targets = torch.from_numpy(targets)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return torch.Tensor(self.inputs[idx]), self.targets[idx]

In [9]:
def trainer(config, train_x, train_y, num_epochs=100, batch_size=32, debug=False, lr=0.0001,model_type="LSTM"):
    train_pos_idx = np.where(train_y==1)
    train_neg_idx = np.where(train_y==0)

    train_xp = train_x[train_pos_idx]
    train_xn = train_x[train_neg_idx]

    train_yp = train_y[train_pos_idx]
    train_yn = train_y[train_neg_idx]

    train_dataset_pos = TrainerDataset(train_xp, train_yp)
    train_dataloader_pos = DataLoader(train_dataset_pos, batch_size=batch_size//2, shuffle=True)
    train_dataset_neg = TrainerDataset(train_xn, train_yn)
    train_dataloader_neg = DataLoader(train_dataset_neg, batch_size=batch_size//2, shuffle=True)

    seed = 12345
    os.environ['PYTHONHASHSEED']=str(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    np.random.seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    model = RNN_Model_Generic(config, model_type).to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),lr=lr)  
    n_total_steps = len(train_dataloader_neg)
    model.train()

    for epoch in range(num_epochs):
        for i, (train_features_neg, train_labels_neg) in enumerate(train_dataloader_neg):
            train_features_pos, train_labels_pos = next(iter(train_dataloader_pos))
            train_features = torch.cat((train_features_pos, train_features_neg),0)
            train_labels = torch.cat((train_labels_pos, train_labels_neg),0)

#             print(train_features.shape, train_labels.shape)

            outputs = model(train_features.to(device))
            loss = criterion(outputs, train_labels.to(device))
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % 2000 == 0 and epoch % 10 == 0:
#             if (i+1) % 200 == 0:
                print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')
                if debug: 
                    return model
    return model

In [10]:
def tester(model, test_x, test_y):
    test_dataset = TrainerDataset(test_x, test_y)
    test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False)
    model.eval()
    results = []
    true_labels = []
    with torch.no_grad():
        for test_features, test_labels in test_dataloader:
            outputs = model(test_features.to(device)).detach().to("cpu")
            results.extend(outputs)
            true_labels.extend(test_labels)
    return true_labels, results

In [11]:
class Stats:
    def __init__(self):
        self.acc = 0
        self.pre = 0
        self.re = 0
        self.f1 = 0
        self.roc = 0
        self.prc = 0
        self.tn = 0
        self.fp = 0
        self.fn = 0
        self.tp = 0
    def print(self):
        print('Accuracy: %.4f' %self.acc)
        print('Precision: %.4f' %self.pre)
        print('Recall: %.4f' %self.re)
        print('F1 Score: %.4f' %self.f1)
        print('ROC: %.4f' %self.roc)
        print('PR AUC: %.4f' %self.prc)
        print("Confusion Matrix")
        print(self.tn, "\t", self.fp)
        print(self.fn, "\t", self.tp)

In [12]:
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score

def eval_matrices(model, test_x, test_y, debug = False):
    true_y, results = tester(model, test_x, test_y)
    predictions = [torch.nn.functional.softmax(r) for r in results]
    pred_y = np.array([y[1].item() for y in predictions])
    pred_y_list = []
    test_y = np.array([y.item() for y in true_y])

    for x in pred_y:
        if(x>0.5):
            pred_y_list.append(1)
        else:
            pred_y_list.append(0)

    pred_y_list = np.array(pred_y_list)

    tn, fp, fn, tp = confusion_matrix(test_y, pred_y_list).ravel()
    precision, recall, _ = precision_recall_curve(test_y, pred_y)
    auc_score = auc(recall, precision)
    acc = accuracy_score(test_y, pred_y_list)

    pr = -1
    re = -1
    f1 = -1
    try:
        pr = tp / (tp+fp)
        re = tp / (tp+fn)
        f1 = 2*pr*re / (pr+re)
    except:
        f1 = -1
    
    stats = Stats()
    stats.acc = acc
    stats.pre = pr
    stats.re = re
    stats.f1 = f1
    stats.roc = roc_auc_score(test_y, pred_y)
    stats.prc = auc_score 
    stats.tn = tn
    stats.fp = fp
    stats.fn = fn
    stats.tp = tp

    if debug:
        print('Accuracy: %.4f' %acc)
        print('Precision: %.4f' %pr)
        print('Recall: %.4f' %re)
        print('F1 Score: %.4f' %f1)
        print('ROC:',roc_auc_score(test_y, pred_y))
        print('PR AUC: %.4f' % auc_score)

        print(classification_report(test_y, pred_y_list, digits=4))
        print("Confusion Matrix")
        print(confusion_matrix(test_y, pred_y_list))
        
    return stats

#One Hot Encoded (4 Channels) Test

##Prepare Data

In [17]:
!pip install pickle5


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


In [13]:
import pickle5 as pkl
in_file = data_dir + "Encoded Data/all_encoded_data_new.pkl"
enc_dict = {}
with open(in_file, "rb") as f:
    enc_dict = pkl.load(f)

In [14]:
data_x = enc_dict['enc_superposed_with_dir']
data_y = enc_dict['labels']

In [15]:
import numpy as np

data_x = np.array(data_x)
data_y = np.array(data_y)

print(data_x.shape)
print(data_y.shape)

(153233, 23, 5)
(153233,)


In [16]:
from sklearn.model_selection import train_test_split

train_xx, test_xr, train_yy, test_yr = train_test_split(data_x, data_y,
                                                    stratify=data_y, 
                                                    test_size=0.20,
                                                    random_state=5)

print(train_xx.shape, train_yy.shape)
print(test_xr.shape, test_yr.shape)

(122586, 23, 5) (122586,)
(30647, 23, 5) (30647,)


In [17]:
train_x, test_x, train_y, test_y = train_test_split(train_xx, train_yy,
                                                    stratify=train_yy, 
                                                    test_size=0.20,
                                                    random_state=5)

print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)

(98068, 23, 5) (98068,)
(24518, 23, 5) (24518,)


#Genetic Algoritm

In [18]:
config= {
    "vocab_size" : 0,
    "emb_size" : 5,
    "hidden_size" : 128,
    "lstm_layers" : 1,
    "bi_lstm" : False, 
    "number_hidder_layers" : 3,
    "dropout_prob" : 0.4,
    "reshape" : False 
}

In [19]:
p_batch_size = [32, 64, 128, 256]
p_epochs = [10, 20, 30, 40, 50 , 60, 70, 80, 90, 100]
p_lstm_layers = [1,2]
p_bi_lstm = [True, False]
p_dropout_prob = [0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50]
p_hidden_size = [32, 64, 128, 256, 512]
p_number_hidden_layers = [2,3,4,5,6]
p_learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]


comb = len(p_epochs) * len(p_lstm_layers) * len(p_bi_lstm)
comb = comb * len(p_dropout_prob) * len(p_hidden_size) * len(p_learning_rates) * len(p_batch_size)
print(comb)

72000


In [20]:
candidate_values = {}

candidate_values["epochs"] = p_epochs
candidate_values["lstm_layers"] = p_lstm_layers
candidate_values["bi_lstm"] = p_bi_lstm
candidate_values["dropout_prob"] = p_dropout_prob
candidate_values["hidden_size"] = p_hidden_size
candidate_values["learning_rate"] = p_learning_rates
candidate_values["batch_size"] = p_batch_size


In [21]:
def generate_random_config():
    config= {
        "vocab_size" : 0,
        "emb_size" : 5,
        "hidden_size" : 128,
        "lstm_layers" : 1,
        "bi_lstm" : False, 
        "number_hidder_layers" : 3,
        "dropout_prob" : 0.4,
        "reshape" : False 
    }

    config["lstm_layers"] = random.choice(p_lstm_layers)
    config["bi_lstm"] = random.choice(p_bi_lstm)
    config["dropout_prob"] = random.choice(p_dropout_prob)
    config["batch_size"] = random.choice(p_batch_size)
    config["hidden_size"] = random.choice(p_hidden_size)

    h_size = config["hidden_size"]
    p_hidden_layers = [0]
    while h_size > 4:
        h_size = h_size//2
        p_hidden_layers.append(p_hidden_layers[-1]+1)
    
    config["number_hidder_layers"] = random.choice(p_hidden_layers) 
    
    config["epochs"] = random.choice(p_epochs)
    config["learning_rate"] = random.choice(p_learning_rates)
    return config

In [22]:
def generate_config_from_log(log, index):
    config= {
        "vocab_size" : 0,
        "emb_size" : 5,
        "hidden_size" : 128,
        "lstm_layers" : 1,
        "bi_lstm" : False, 
        "number_hidder_layers" : 3,
        "dropout_prob" : 0.4,
        "reshape" : False 
    }

    config["lstm_layers"] = int(log.iloc[index]["lstm_layers"])
    config["bi_lstm"] =  True if log.iloc[index]["bi_lstm"] == "TRUE" else False
    config["dropout_prob"] = float(log.iloc[index]["dropout_prob"])
    config["batch_size"] = int(log.iloc[index]["batch_size"])
    config["hidden_size"] = int(log.iloc[index]["hidden_size"])
    config["number_hidder_layers"] = int(log.iloc[index]["number_hidder_layers"])
    config["epochs"] = int(log.iloc[index]["epochs"])
    config["learning_rate"] = float(log.iloc[index]["learning_rate"])
    
    return config

In [23]:
class Individual():
    def __init__(self):
        self.config = {}
        self.stat = Stats()
        self.trained = False
    def fitness(self):
        if self.stat.tp == 0:
            return 0
        return self.stat.prc
    
    def __lt__(self, obj):
        return ((self.fitness()) < (obj.fitness()))
  
    def __gt__(self, obj):
        return ((self.fitness()) > (obj.fitness()))
  
    def __le__(self, obj):
        return ((self.fitness()) <= (obj.fitness()))
  
    def __ge__(self, obj):
        return ((self.fitness()) >= (obj.fitness()))
  
    def __eq__(self, obj):
        return ((self.fitness()) == (obj.fitness()))

In [24]:
def generate_init_population(popsize = 10, debug=False):
    population = []
    for i in range(popsize):
        indiv = Individual()
        indiv.config = generate_random_config()
        population.append(indiv)
    
    if debug:
        for p in population:
            print(p.config)
    
    return population

In [25]:
import pandas as pd

def read_last_generation(popsize, log_filename, cont, debug=False):
    log = pd.read_csv(log_filename)
    population = []
    
    for i in range(cont*popsize, (cont+1)*popsize, 1):
        indiv = Individual()
        indiv.config = generate_config_from_log(log, i)
        indiv.trained = True
        
        indiv.stat.acc = log.iloc[i]["acc"]
        indiv.stat.pre = log.iloc[i]["pre"]
        indiv.stat.re = log.iloc[i]["re"]
        indiv.stat.f1 = log.iloc[i]["f1"]
        indiv.stat.roc = log.iloc[i]["roc"]
        indiv.stat.prc = log.iloc[i]["prc"]
        indiv.stat.tn = log.iloc[i]["tn"]
        indiv.stat.fp = log.iloc[i]["fp"]
        indiv.stat.fn = log.iloc[i]["fn"]
        indiv.stat.tp = log.iloc[i]["tp"]
        population.append(indiv)
        
    
    if debug:
        for p in population:
            print(p.config)
    
    return population

In [26]:
def train_model(config, debug=False, model_type="LSTM"):
    if debug:
       config["epochs"] = 1
    try:
        model = trainer(config, train_x, train_y, num_epochs=config["epochs"], lr=config["learning_rate"], batch_size = int(config["batch_size"]), model_type = model_type)
        stats = eval_matrices(model, test_x, test_y)
        return stats
    except:
        print(config["batch_size"])
        print("Should Exit Here")

In [27]:
def generate_fitness_score(population, debug = False, model_type="LSTM"):
    for p in population:
        print(p.config)
        if p.trained == True:
            print("Copied Elite. Already Trained.")
        else:
            stat = train_model(p.config, debug, model_type)
            p.stat = stat
        print(p.stat.print())
    return population

In [28]:
def assess_fitness(population, best):
    for i in range(len(population)):
        p = population[i]
        if best is None or best.fitness() < p.fitness():
            best = Individual()
            best = copy.deepcopy(p)
    return best

In [29]:
def tournament_selection(population, t_size = 2):
    best = random.choice(population) 
    for t in range(t_size):
        next = random.choice(population)
        if next.fitness() > best.fitness():
            best = next
    return next   

In [30]:
def crossover(indiv1, indiv2):
    child1 = copy.deepcopy(indiv1)
    child1.trained = False
    child2 = copy.deepcopy(indiv2)
    child2.trained = False
    fixed_keys = ["vocab_size","emb_size","number_hidder_layers","reshape"]
    keys = child1.config.keys()
    for k in keys:
        if k not in fixed_keys:
            if random.random() < 0.3:
                temp = indiv1.config[k]
                child1.config[k] = child2.config[k]
                child2.config[k] = temp

                if k == "hidden_size":
                    k1 = "number_hidder_layers"
                    temp = child1.config[k1]
                    child1.config[k1] = child2.config[k1]
                    child2.config[k1] = temp
                    

    return child1, child2

In [31]:
def mutate(indiv):
    fixed_keys = ["vocab_size","emb_size","number_hidder_layers","reshape"]
    keys = indiv.config.keys()
    for k in keys:
        if k not in fixed_keys:
            if random.random() < 0.2:
                temp = indiv.config[k]
                indiv.config[k] = random.choice(candidate_values[k])

                if k == "hidden_size":
                    h_size = indiv.config[k]
                    p_hidden_layers = [0]
                    while h_size > 4:
                        h_size = h_size//2
                        p_hidden_layers.append(p_hidden_layers[-1]+1)
                    k1 = "number_hidder_layers"
                    indiv.config[k1] = random.choice(p_hidden_layers) 

    indiv.trained = False
    return indiv

In [32]:
import csv 

def plain_genetic_algorithm(pop_size=4, itr=2, debug=True, t_size = 2, model_type="LSTM", ratio=1, cont = -1):
#     population = generate_init_population(pop_size)
#     keys = population[0].config.keys()
    
#     fields = ["Itr", "vocab_size" ,"emb_size" ,"hidden_size", "lstm_layers" ,"bi_lstm","number_hidder_layers"]
#     fields += ["dropout_prob", "reshape", "batch_size","epochs", "learning_rate"]
    
# #     for k in keys:
# #         fields.append(k)
    
#     fields += ["acc","pre","re","f1","roc","prc","tn","fp","fn","tp"]

#     log_filename = "ga_stat/"+ model_type + "-" + str(ratio) +"-log-" + str(pop_size) + "-" + str(itr) + "-" +str(t_size) +".csv"
#     logfile = open(log_filename,"w")
#     csvwriter = csv.writer(logfile)
#     csvwriter.writerow(fields)
#     logfile.flush()
#     print(fields)  

#     best = None

#     if debug:
#         print("Running in debug mode.")
    
#     print("Iteration ",0, "running...")
#     population = generate_fitness_score(population, debug, model_type=model_type)
#     best = assess_fitness(population, best)
    
#     for p in population:
#         fields = [0]
#         for k in keys:
#             fields.append(p.config[k])
#         s = p.stat
#         fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
#         csvwriter.writerow(fields)
#         print(fields)
#     logfile.flush()
    log_filename = "ga_stat/"+ model_type + "-" + str(ratio) +"-log5ch-" 
    log_filename = log_filename + str(pop_size) + "-" + str(itr) + "-" +str(t_size) +".csv"
    
    if cont>=0:    
        print("Reading Last Checkpoint")
        population = read_last_generation(pop_size, log_filename, cont)
        
    else:
        population = generate_init_population(pop_size)
    
    keys = population[0].config.keys()
    
    fields = ["Itr"]
    for k in keys:
        fields.append(k)
    fields += ["acc","pre","re","f1","roc","prc","tn","fp","fn","tp"]
    
    
    if cont>=0:
        logfile = open(log_filename,"a")
        csvwriter = csv.writer(logfile)
        
    else:
        logfile = open(log_filename,"w")
        csvwriter = csv.writer(logfile)
        csvwriter.writerow(fields)
        logfile.flush()
    
    i = max(0, cont)
    
    print(fields)
    if debug:
        print("Running in debug mode.")
        
    print("Iteration ",i, "running...")
    population = generate_fitness_score(population, debug, model_type=model_type)  

    best = None
    
    sorted_population = sorted(population, reverse=True)
    best = assess_fitness(population, best)
    
    for p in population:
        fields = [i]
        for k in keys:
            fields.append(p.config[k])
        s = p.stat
        fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
        if cont < 0:
            csvwriter.writerow(fields)
        print(fields)
    logfile.flush()
    

    while i < itr:
        Q = []
        for j in range(pop_size//2):
            individual1 = tournament_selection(population, t_size)
            individual2 = tournament_selection(population, t_size)
            child1, child2 = crossover(individual1, individual2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            Q.append(child1)
            Q.append(child2)

        population = Q

        print("Iteration ",(i+1), "running...")
        population = generate_fitness_score(population, debug)
        best = assess_fitness(population, best)

        for p in population:
            fields = [i+1]
            for k in keys:
                fields.append(p.config[k])
            s = p.stat
            fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
            csvwriter.writerow(fields)
            print(fields)
        logfile.flush()
        i += 1

    fields = [itr+1]
    for k in keys:
        fields.append(best.config[k])
    s = best.stat
    fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
    csvwriter.writerow(fields)
    print(fields)

    logfile.close()
    return best

In [ ]:
# best = plain_genetic_algorithm(20,10,False,2, model_type="RNN", ratio=1, cont=0)

Reading Last Checkpoint
['Itr', 'vocab_size', 'emb_size', 'hidden_size', 'lstm_layers', 'bi_lstm', 'number_hidder_layers', 'dropout_prob', 'reshape', 'batch_size', 'epochs', 'learning_rate', 'acc', 'pre', 're', 'f1', 'roc', 'prc', 'tn', 'fp', 'fn', 'tp']
Iteration  0 running...
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 64, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.3, 'reshape': False, 'batch_size': 128, 'epochs': 100, 'learning_rate': 0.01}
Copied Elite. Already Trained.
Accuracy: 0.4560
Precision: 0.0078
Recall: 1.0000
F1 Score: 0.0155
ROC: 0.7217
PR AUC: 0.1304
Confusion Matrix
11075 	 13338
0 	 105
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 4, 'dropout_prob': 0.15, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.5}
Copied Elite. Already Trained.
Accuracy: 0.0043
Precision: 0.0043
Recall: 1.0000
F1 Score: 0.0085
ROC: 0.5000
PR AUC: 0.5021
Confu

Epoch [1/10], Step [2000/3052], Loss: 0.1496


/tmp/ipykernel_2855/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9922
Precision: 0.3320
Recall: 0.8190
F1 Score: 0.4725
ROC: 0.9922
PR AUC: 0.6169
Confusion Matrix
24240 	 173
19 	 86
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 0, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 256, 'epochs': 20, 'learning_rate': 1e-05}


/tmp/ipykernel_2855/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


...

Accuracy: 0.0043
Precision: 0.0043
Recall: 1.0000
F1 Score: 0.0085
ROC: 0.5000
PR AUC: 0.5021
Confusion Matrix
0 	 24413
0 	 105
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 32, 'epochs': 30, 'learning_rate': 0.0001}
Epoch [1/30], Step [2000/6103], Loss: 0.1131
Epoch [1/30], Step [4000/6103], Loss: 0.0532
Epoch [1/30], Step [6000/6103], Loss: 0.2138
Epoch [11/30], Step [2000/6103], Loss: 0.2238
Epoch [11/30], Step [4000/6103], Loss: 0.0003
Epoch [11/30], Step [6000/6103], Loss: 0.0036
Epoch [21/30], Step [2000/6103], Loss: 0.0052
Epoch [21/30], Step [4000/6103], Loss: 0.0003
Epoch [21/30], Step [6000/6103], Loss: 0.0003


/tmp/ipykernel_2855/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9959
Precision: 0.5191
Recall: 0.6476
F1 Score: 0.5763
ROC: 0.9892
PR AUC: 0.6478
Confusion Matrix
24350 	 63
37 	 68
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 32, 'epochs': 30, 'learning_rate': 0.0005}
Epoch [1/30], Step [2000/6103], Loss: 0.0178
Epoch [1/30], Step [4000/6103], Loss: 0.1010
Epoch [1/30], Step [6000/6103], Loss: 0.1484
Epoch [11/30], Step [2000/6103], Loss: 0.0001
Epoch [11/30], Step [4000/6103], Loss: 0.0000
Epoch [11/30], Step [6000/6103], Loss: 0.0000
Epoch [21/30], Step [2000/6103], Loss: 0.0013
Epoch [21/30], Step [4000/6103], Loss: 0.0000
Epoch [21/30], Step [6000/6103], Loss: 0.0000


/tmp/ipykernel_2855/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9973
Precision: 0.7794
Recall: 0.5048
F1 Score: 0.6127
ROC: 0.9503
PR AUC: 0.6469
Confusion Matrix
24398 	 15
52 	 53
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 1, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 32, 'epochs': 90, 'learning_rate': 0.0001}
Epoch [1/90], Step [2000/6103], Loss: 0.5145
Epoch [1/90], Step [4000/6103], Loss: 0.0291
Epoch [1/90], Step [6000/6103], Loss: 0.1307
Epoch [11/90], Step [2000/6103], Loss: 0.0268
Epoch [11/90], Step [4000/6103], Loss: 0.0033
Epoch [11/90], Step [6000/6103], Loss: 0.0041
Epoch [21/90], Step [2000/6103], Loss: 0.0054
Epoch [21/90], Step [4000/6103], Loss: 0.0015
Epoch [21/90], Step [6000/6103], Loss: 0.0082


In [36]:
best = plain_genetic_algorithm(20,10,False,2, model_type="LSTM", ratio=1, cont=5)

Reading Last Checkpoint
['Itr', 'vocab_size', 'emb_size', 'hidden_size', 'lstm_layers', 'bi_lstm', 'number_hidder_layers', 'dropout_prob', 'reshape', 'batch_size', 'epochs', 'learning_rate', 'acc', 'pre', 're', 'f1', 'roc', 'prc', 'tn', 'fp', 'fn', 'tp']
Iteration  5 running...
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 1, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 64, 'epochs': 80, 'learning_rate': 0.005}
Copied Elite. Already Trained.
Accuracy: 0.9924
Precision: 0.3248
Recall: 0.7238
F1 Score: 0.4484
ROC: 0.9791
PR AUC: 0.4182
Confusion Matrix
24255 	 158
29 	 76
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 1, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.0001}
Copied Elite. Already Trained.
Accuracy: 0.9955
Precision: 0.4792
Recall: 0.6571
F1 Score: 0.5542
ROC: 0.9832
PR AUC: 0.6236
Confus

Epoch [1/80], Step [2000/3052], Loss: 0.0582
Epoch [11/80], Step [2000/3052], Loss: 0.0680
Epoch [21/80], Step [2000/3052], Loss: 0.0193
Epoch [31/80], Step [2000/3052], Loss: 0.0200
Epoch [41/80], Step [2000/3052], Loss: 0.0181
Epoch [51/80], Step [2000/3052], Loss: 0.0035
Epoch [61/80], Step [2000/3052], Loss: 0.0057
Epoch [71/80], Step [2000/3052], Loss: 0.0437


/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9849
Precision: 0.1696
Recall: 0.6476
F1 Score: 0.2688
ROC: 0.9746
PR AUC: 0.2467
Confusion Matrix
24080 	 333
37 	 68
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 32, 'epochs': 20, 'learning_rate': 0.0001}
Epoch [1/20], Step [2000/6103], Loss: 0.1905
Epoch [1/20], Step [4000/6103], Loss: 0.0436
Epoch [1/20], Step [6000/6103], Loss: 0.0201
Epoch [11/20], Step [2000/6103], Loss: 0.0004
Epoch [11/20], Step [4000/6103], Loss: 0.0015
Epoch [11/20], Step [6000/6103], Loss: 0.0013


/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


...

/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9961
Precision: 0.5391
Recall: 0.6571
F1 Score: 0.5923
ROC: 0.9838
PR AUC: 0.5933
Confusion Matrix
24354 	 59
36 	 69
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 32, 'epochs': 80, 'learning_rate': 0.0001}
Epoch [1/80], Step [2000/6103], Loss: 0.1583
Epoch [1/80], Step [4000/6103], Loss: 0.1122
Epoch [1/80], Step [6000/6103], Loss: 0.0701
Epoch [11/80], Step [2000/6103], Loss: 0.0016
Epoch [11/80], Step [4000/6103], Loss: 0.0028
Epoch [11/80], Step [6000/6103], Loss: 0.0010
Epoch [21/80], Step [2000/6103], Loss: 0.0006
Epoch [21/80], Step [4000/6103], Loss: 0.0001
Epoch [21/80], Step [6000/6103], Loss: 0.0001
Epoch [31/80], Step [2000/6103], Loss: 0.0002
Epoch [31/80], Step [4000/6103], Loss: 0.0001
Epoch [31/80], Step [6000/6103], Loss: 0.0002
Epoch [41/80], Step [2000/6103], Loss: 0.0000
Epoch [41/80], Step [4000/6103], Loss: 0.0000
Epoch [41/80

/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9969
Precision: 0.6706
Recall: 0.5429
F1 Score: 0.6000
ROC: 0.9825
PR AUC: 0.6271
Confusion Matrix
24385 	 28
48 	 57
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 256, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 256, 'epochs': 90, 'learning_rate': 0.1}
Accuracy: 0.0043
Precision: 0.0043
Recall: 1.0000
F1 Score: 0.0085
ROC: 0.5000
PR AUC: 0.5021
Confusion Matrix
0 	 24413
0 	 105
None
[10, 0, 5, 512, 1, False, 2, 0.1, False, 64, 80, 0.005, 0.9849090464148789, 0.16957605985037408, 0.6476190476190476, 0.26877470355731226, 0.9745810682442805, 0.24670443366722938, 24080, 333, 37, 68]
[10, 0, 5, 512, 2, False, 2, 0.2, False, 32, 20, 0.0001, 0.9952687821192593, 0.46405228758169936, 0.6761904761904762, 0.5503875968992249, 0.9860160375131907, 0.5988590254323244, 24331, 82, 34, 71]
[10, 0, 5, 256, 1, True, 2, 0.1, False, 32, 30, 1e-05, 0.9813198466432825, 0.166351606805293, 0.8380952380952381, 0.277602523

/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


In [34]:
# best = plain_genetic_algorithm(20,10,False,2, model_type="GRU", ratio=1, cont=0)

Reading Last Checkpoint
['Itr', 'vocab_size', 'emb_size', 'hidden_size', 'lstm_layers', 'bi_lstm', 'number_hidder_layers', 'dropout_prob', 'reshape', 'batch_size', 'epochs', 'learning_rate', 'acc', 'pre', 're', 'f1', 'roc', 'prc', 'tn', 'fp', 'fn', 'tp']
Iteration  0 running...
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 64, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.3, 'reshape': False, 'batch_size': 128, 'epochs': 100, 'learning_rate': 0.01}
Copied Elite. Already Trained.
Accuracy: 0.9593
Precision: 0.0892
Recall: 0.9238
F1 Score: 0.1626
ROC: 0.9859
PR AUC: 0.3422
Confusion Matrix
23422 	 991
8 	 97
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 4, 'dropout_prob': 0.15, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.5}
Copied Elite. Already Trained.
Accuracy: 0.0043
Precision: 0.0043
Recall: 1.0000
F1 Score: 0.0085
ROC: 0.5000
PR AUC: 0.5021
Confusio

Epoch [1/10], Step [2000/3052], Loss: 0.1716


/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9960
Precision: 0.5271
Recall: 0.6476
F1 Score: 0.5812
ROC: 0.9908
PR AUC: 0.6418
Confusion Matrix
24352 	 61
37 	 68
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 0, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 256, 'epochs': 20, 'learning_rate': 1e-05}


/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


...

/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9936
Precision: 0.3857
Recall: 0.8190
F1 Score: 0.5244
ROC: 0.9910
PR AUC: 0.6792
Confusion Matrix
24276 	 137
19 	 86
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 256, 'lstm_layers': 1, 'bi_lstm': True, 'number_hidder_layers': 2, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 32, 'epochs': 30, 'learning_rate': 0.0001}
Epoch [1/30], Step [2000/6103], Loss: 0.2513
Epoch [1/30], Step [4000/6103], Loss: 0.0253
Epoch [1/30], Step [6000/6103], Loss: 0.0148
Epoch [11/30], Step [2000/6103], Loss: 0.0051
Epoch [11/30], Step [4000/6103], Loss: 0.0019
Epoch [11/30], Step [6000/6103], Loss: 0.0039
Epoch [21/30], Step [2000/6103], Loss: 0.0003
Epoch [21/30], Step [4000/6103], Loss: 0.0004
Epoch [21/30], Step [6000/6103], Loss: 0.0006


/tmp/ipykernel_21306/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9956
Precision: 0.4880
Recall: 0.7714
F1 Score: 0.5978
ROC: 0.9867
PR AUC: 0.6767
Confusion Matrix
24328 	 85
24 	 81
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 256, 'lstm_layers': 1, 'bi_lstm': True, 'number_hidder_layers': 2, 'dropout_prob': 0.2, 'reshape': False, 'batch_size': 32, 'epochs': 90, 'learning_rate': 1e-05}
Epoch [1/90], Step [2000/6103], Loss: 0.3864
Epoch [1/90], Step [4000/6103], Loss: 0.2947
Epoch [1/90], Step [6000/6103], Loss: 0.3053
Epoch [11/90], Step [2000/6103], Loss: 0.0445
Epoch [11/90], Step [4000/6103], Loss: 0.1334
Epoch [11/90], Step [6000/6103], Loss: 0.1102
Epoch [21/90], Step [2000/6103], Loss: 0.0094
Epoch [21/90], Step [4000/6103], Loss: 0.0143
Epoch [21/90], Step [6000/6103], Loss: 0.0235
Epoch [31/90], Step [2000/6103], Loss: 0.0071
Epoch [31/90], Step [4000/6103], Loss: 0.1199
Epoch [31/90], Step [6000/6103], Loss: 0.0467
Epoch [41/90], Step [2000/6103], Loss: 0.0508
Epoch [41/90], Step [4000/6103], Loss: 0.0952
Epoch [41/90],

AttributeError: 'NoneType' object has no attribute 'print'

##Test

In [34]:
# print(best.config)
# best.stat.print()

In [35]:
import csv 

def elitist_genetic_algorithm(pop_size=4,  itr=2, debug=True, t_size = 2, model_type="LSTM", 
                              ratio=1, elite_size = 2, cont=-1):
    
    log_filename = "ega_stat/"+ model_type + "-" + str(ratio) +"-log5ch-" 
    log_filename = log_filename + str(pop_size) + "-" + str(itr) + "-" +str(t_size) +".csv"
    
    if cont>=0:    
        print("Reading Last Checkpoint")
        population = read_last_generation(pop_size, log_filename, cont)
        
    else:
        population = generate_init_population(pop_size)
    
    keys = population[0].config.keys()
    
    fields = ["Itr"]
    for k in keys:
        fields.append(k)
    fields += ["acc","pre","re","f1","roc","prc","tn","fp","fn","tp"]
    
    
    if cont>=0:
        logfile = open(log_filename,"a")
        csvwriter = csv.writer(logfile)
        
    else:
        logfile = open(log_filename,"w")
        csvwriter = csv.writer(logfile)
        csvwriter.writerow(fields)
        logfile.flush()
    
    i = max(0, cont)
    
    print(fields)
    if debug:
        print("Running in debug mode.")
        
    print("Iteration ",i, "running...")
    population = generate_fitness_score(population, debug, model_type=model_type)  

    best = None
    
    sorted_population = sorted(population, reverse=True)
    best = assess_fitness(population, best)
    
    for p in population:
        fields = [i]
        for k in keys:
            fields.append(p.config[k])
        s = p.stat
        fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
        if cont < 0:
            csvwriter.writerow(fields)
        print(fields)
    logfile.flush()
    
    while i < itr:
        Q = []
        for j in range(elite_size):
            elite = copy.deepcopy(sorted_population[j])
            elite.trained = True
            Q.append(elite)
            
        for j in range((pop_size-elite_size)//2):
            individual1 = tournament_selection(population, t_size)
            individual2 = tournament_selection(population, t_size)
            child1, child2 = crossover(individual1, individual2)
            child1 = mutate(child1)
            child2 = mutate(child2)
            Q.append(child1)
            Q.append(child2)

        population = Q

        print("Iteration ",(i+1), "running...")
        population = generate_fitness_score(population, debug)
        sorted_population = sorted(population, reverse=True)
        best = assess_fitness(population, best)

        for p in population:
            fields = [i+1]
            for k in keys:
                fields.append(p.config[k])
            s = p.stat
            fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
            csvwriter.writerow(fields)
            print(fields)
        logfile.flush()  
        i = i + 1
        

    fields = [itr+1]
    for k in keys:
        fields.append(best.config[k])
    s = best.stat
    fields +=  [s.acc,s.pre,s.re,s.f1,s.roc,s.prc,s.tn,s.fp,s.fn,s.tp]
    csvwriter.writerow(fields)
    print(fields)

    logfile.close()
    return best

In [36]:
# best = elitist_genetic_algorithm(20,10,False,2, model_type="RNN", ratio=1, elite_size=4, cont=6)

In [39]:
# print(best.config)
# best.stat.print()

In [37]:
# best = elitist_genetic_algorithm(20,10,False,2, model_type="GRU", ratio=1, elite_size=4, cont=-1)

['Itr', 'vocab_size', 'emb_size', 'hidden_size', 'lstm_layers', 'bi_lstm', 'number_hidder_layers', 'dropout_prob', 'reshape', 'batch_size', 'epochs', 'learning_rate', 'acc', 'pre', 're', 'f1', 'roc', 'prc', 'tn', 'fp', 'fn', 'tp']
Iteration  0 running...
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 64, 'lstm_layers': 2, 'bi_lstm': True, 'number_hidder_layers': 2, 'dropout_prob': 0.3, 'reshape': False, 'batch_size': 128, 'epochs': 100, 'learning_rate': 0.01}


/tmp/ipykernel_17124/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9607
Precision: 0.0890
Recall: 0.8857
F1 Score: 0.1617
ROC: 0.9703
PR AUC: 0.4598
Confusion Matrix
23461 	 952
12 	 93
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 4, 'dropout_prob': 0.15, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.5}


/tmp/ipykernel_17124/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.0043
Precision: 0.0043
Recall: 1.0000
F1 Score: 0.0085
ROC: 0.5000
PR AUC: 0.5021
Confusion Matrix
0 	 24413
0 	 105
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.15, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.0005}


...

/tmp/ipykernel_17124/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9695
Precision: 0.1103
Recall: 0.8667
F1 Score: 0.1957
ROC: 0.9822
PR AUC: 0.3953
Confusion Matrix
23679 	 734
14 	 91
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 256, 'epochs': 80, 'learning_rate': 1e-05}


/tmp/ipykernel_17124/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9949
Precision: 0.4379
Recall: 0.7048
F1 Score: 0.5401
ROC: 0.9817
PR AUC: 0.6112
Confusion Matrix
24318 	 95
31 	 74
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 256, 'epochs': 80, 'learning_rate': 0.0005}
Accuracy: 0.9955
Precision: 0.4806
Recall: 0.5905
F1 Score: 0.5299
ROC: 0.9715
PR AUC: 0.5161
Confusion Matrix
24346 	 67
43 	 62
None
[10, 0, 5, 512, 2, False, 2, 0.1, False, 256, 80, 0.001, 0.9974712456154662, 0.8115942028985508, 0.5333333333333333, 0.6436781609195403, 0.9889699672110683, 0.6951219446018551, 24400, 13, 49, 56]
[10, 0, 5, 512, 2, False, 2, 0.1, False, 256, 80, 0.001, 0.9974712456154662, 0.8115942028985508, 0.5333333333333333, 0.6436781609195403, 0.9889699672110683, 0.6951219446018551, 24400, 13, 49, 56]
[10, 0, 5, 512, 2, False, 2, 0.1, False, 256, 80, 0.001, 0.9974712456154662, 0.8115942028985508, 0.5333333333333333, 0.643678

/tmp/ipykernel_17124/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


In [120]:
# print(best.config)
# best.stat.print()

{'vocab_size': 0, 'emb_size': 4, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.35, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.0001}
Accuracy: 0.9947
Precision: 0.4343
Recall: 0.8190
F1 Score: 0.5677
ROC: 0.9912
PR AUC: 0.7190
Confusion Matrix
24301 	 112
19 	 86


In [36]:
best = elitist_genetic_algorithm(20,10,False,2, model_type="LSTM", ratio=1, elite_size=4, cont=-1)

['Itr', 'vocab_size', 'emb_size', 'hidden_size', 'lstm_layers', 'bi_lstm', 'number_hidder_layers', 'dropout_prob', 'reshape', 'batch_size', 'epochs', 'learning_rate', 'acc', 'pre', 're', 'f1', 'roc', 'prc', 'tn', 'fp', 'fn', 'tp']
Iteration  0 running...
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 64, 'lstm_layers': 2, 'bi_lstm': True, 'number_hidder_layers': 2, 'dropout_prob': 0.3, 'reshape': False, 'batch_size': 128, 'epochs': 100, 'learning_rate': 0.01}


/tmp/ipykernel_27447/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9809
Precision: 0.1527
Recall: 0.7619
F1 Score: 0.2544
ROC: 0.9765
PR AUC: 0.4591
Confusion Matrix
23969 	 444
25 	 80
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 128, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 4, 'dropout_prob': 0.15, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.5}


/tmp/ipykernel_27447/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.0043
Precision: 0.0043
Recall: 1.0000
F1 Score: 0.0085
ROC: 0.5000
PR AUC: 0.5021
Confusion Matrix
0 	 24413
0 	 105
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.15, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.0005}


...

/tmp/ipykernel_27447/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9695
Precision: 0.1103
Recall: 0.8667
F1 Score: 0.1957
ROC: 0.9822
PR AUC: 0.3953
Confusion Matrix
23679 	 734
14 	 91
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 256, 'epochs': 80, 'learning_rate': 1e-05}


/tmp/ipykernel_27447/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


Accuracy: 0.9949
Precision: 0.4379
Recall: 0.7048
F1 Score: 0.5401
ROC: 0.9817
PR AUC: 0.6112
Confusion Matrix
24318 	 95
31 	 74
None
{'vocab_size': 0, 'emb_size': 5, 'hidden_size': 512, 'lstm_layers': 2, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.1, 'reshape': False, 'batch_size': 256, 'epochs': 80, 'learning_rate': 0.0005}
Accuracy: 0.9955
Precision: 0.4806
Recall: 0.5905
F1 Score: 0.5299
ROC: 0.9715
PR AUC: 0.5161
Confusion Matrix
24346 	 67
43 	 62
None
[10, 0, 5, 512, 2, False, 2, 0.1, False, 256, 80, 0.001, 0.9974712456154662, 0.8115942028985508, 0.5333333333333333, 0.6436781609195403, 0.9889699672110683, 0.6951219446018551, 24400, 13, 49, 56]
[10, 0, 5, 512, 2, False, 2, 0.1, False, 256, 80, 0.001, 0.9974712456154662, 0.8115942028985508, 0.5333333333333333, 0.6436781609195403, 0.9889699672110683, 0.6951219446018551, 24400, 13, 49, 56]
[10, 0, 5, 512, 2, False, 2, 0.1, False, 256, 80, 0.001, 0.9974712456154662, 0.8115942028985508, 0.5333333333333333, 0.643678

/tmp/ipykernel_27447/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


In [123]:
print(best.config)
best.stat.print()

{'vocab_size': 0, 'emb_size': 4, 'hidden_size': 512, 'lstm_layers': 1, 'bi_lstm': False, 'number_hidder_layers': 2, 'dropout_prob': 0.35, 'reshape': False, 'batch_size': 256, 'epochs': 30, 'learning_rate': 0.0001}
Accuracy: 0.9947
Precision: 0.4343
Recall: 0.8190
F1 Score: 0.5677
ROC: 0.9912
PR AUC: 0.7190
Confusion Matrix
24301 	 112
19 	 86


In [ ]:
# https://c57e-103-94-135-141.ngrok-free.app

## Best Model

In [22]:
#Best RNN Model

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 256, 
#     'lstm_layers': 2, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 0, 
#     'dropout_prob': 0.4, 
#     'reshape': False, 
#     'batch_size': 256, 
#     'epochs': 60, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9816
# Precision: 0.1790
# Recall: 0.9237
# F1 Score: 0.2999
# ROC: 0.9899
# PR AUC: 0.5633
# Confusion Matrix
# 29961 	 555
# 10 	 121
0.9816
# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 128, 
#     'lstm_layers': 1, 
#     'bi_lstm': False, 
#     'number_hidder_layers': 4, 
#     'dropout_prob': 0.2, 
#     'reshape': False, 
#     'batch_size': 128, 
#     'epochs': 30, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9840
# Precision: 0.1970
# Recall: 0.8931
# F1 Score: 0.3228
# ROC: 0.9892
# PR AUC: 0.5851
# Confusion Matrix
# 30039 	 477
# 14 	 117

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 128, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 4, 
#     'dropout_prob': 0.15, 
#     'reshape': False, 
#     'batch_size': 128, 
#     'epochs': 30, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9944
# Precision: 0.4187
# Recall: 0.7863
# F1 Score: 0.5464
# ROC: 0.9910
# PR AUC: 0.6644
# Confusion Matrix
# 30373 	 143
# 28 	 103

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 128, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.2, 
#     'reshape': False, 
#     'batch_size': 128, 
#     'epochs': 30, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9944
# Precision: 0.4163
# Recall: 0.7786
# F1 Score: 0.5426
# ROC: 0.9908
# PR AUC: 0.6293
# Confusion Matrix
# 30373 	 143
# 29 	 102

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 256, 
#     'lstm_layers': 2, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 3, 
#     'dropout_prob': 0.35, 
#     'reshape': False, 
#     'batch_size': 256, 
#     'epochs': 100, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.8360
# Precision: 0.0243
# Recall: 0.9542
# F1 Score: 0.0474
# ROC: 0.9701
# PR AUC: 0.1546
# Confusion Matrix
# 25495 	 5021
# 6 	 125

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 128, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.35, 
#     'reshape': False, 
#     'batch_size': 128, 
#     'epochs': 100, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9579
# Precision: 0.0786
# Recall: 0.8244
# F1 Score: 0.1435
# ROC: 0.9685
# PR AUC: 0.2588
# Confusion Matrix
# 29250 	 1266
# 23 	 108

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 256, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.2, 
#     'reshape': False, 
#     'batch_size': 256, 
#     'epochs': 70, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9890
# Precision: 0.2488
# Recall: 0.7786
# F1 Score: 0.3771
# ROC: 0.9842
# PR AUC: 0.4843
# Confusion Matrix
# 30208 	 308
# 29 	 102

In [43]:
#Best GRU Model

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 512, 
#     'lstm_layers': 1, 
#     'bi_lstm': False, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.4, 
#     'reshape': False, 
#     'batch_size': 64, 
#     'epochs': 50, 
#     'learning_rate': 0.0001
# }

# Accuracy: 0.9969
# Precision: 0.6364
# Recall: 0.6412
# F1 Score: 0.6388
# ROC: 0.9893
# PR AUC: 0.6899
# Confusion Matrix
# 30468 	 48
# 47 	 84

best_config = {
    'vocab_size': 0, 
    'emb_size': 4, 
    'hidden_size': 128, 
    'lstm_layers': 1, 
    'bi_lstm': False, 
    'number_hidder_layers': 4, 
    'dropout_prob': 0.2, 
    'reshape': False, 
    'batch_size': 64, 
    'epochs': 50,
    'learning_rate': 0.001
}

# Accuracy: 0.9969
# Precision: 0.6364
# Recall: 0.6412
# F1 Score: 0.6388
# ROC: 0.9893
# PR AUC: 0.6899
# Confusion Matrix
# 30468 	 48
# 47 	 84

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 512, 
#     'lstm_layers': 2,
#     'bi_lstm': False, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.1,
#     'reshape': False, 
#     'batch_size': 64, 
#     'epochs': 50, 
#     'learning_rate': 0.00005
# }

# Accuracy: 0.9968
# Precision: 0.6039
# Recall: 0.7099
# F1 Score: 0.6526
# ROC: 0.9894
# PR AUC: 0.6994
# Confusion Matrix
# 30455 	 61
# 38 	 93

In [39]:
# Best LSTM Model

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 512, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.4, 
#     'reshape': False, 
#     'batch_size': 64, 
#     'epochs': 50, 
#     'learning_rate': 0.0001
# }

# Accuracy: 0.9971
# Precision: 0.6810
# Recall: 0.6031
# F1 Score: 0.6397
# ROC: 0.9889
# PR AUC: 0.6980
# Confusion Matrix
# 30479 	 37
# 52 	 79

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 512, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 2, 
#     'dropout_prob': 0.1, 
#     'reshape': False, 
#     'batch_size': 256, 
#     'epochs': 80, 
#     'learning_rate': 5.00E-05
# }

# Accuracy: 0.9958
# Precision: 0.5023
# Recall: 0.8397
# F1 Score: 0.6286
# ROC: 0.9917
# PR AUC: 0.6962

# best_config = {
#     'vocab_size': 0, 
#     'emb_size': 4, 
#     'hidden_size': 64, 
#     'lstm_layers': 1, 
#     'bi_lstm': True, 
#     'number_hidder_layers': 4, 
#     'dropout_prob': 0.15, 
#     'reshape': False, 
#     'batch_size': 256, 
#     'epochs': 30, 
#     'learning_rate': 0.001
# }

# Accuracy: 0.9947
# Precision: 0.4362
# Recall: 0.8092
# F1 Score: 0.5668
# ROC: 0.9891
# PR AUC: 0.6561

In [44]:
import pickle5 as pkl
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

in_file = data_dir + "Encoded Data/all_encoded_data.pkl"
enc_dict = {}
with open(in_file, "rb") as f:
    enc_dict = pkl.load(f)

data_x = enc_dict['enc_superposed']
data_y = enc_dict['labels']

data_x = np.array(data_x)
data_y = np.array(data_y)

print(data_x.shape)
print(data_y.shape)


train_x, test_x, train_y, test_y = train_test_split(data_x, data_y,
                                                    stratify=data_y, 
                                                    test_size=0.20,
                                                    random_state=5)
print(train_x.shape, train_y.shape)
print(test_x.shape, test_y.shape)


(153233, 23, 4)
(153233,)
(122586, 23, 4) (122586,)
(30647, 23, 4) (30647,)


In [46]:
config = best_config
debug=False
model_type="GRU"

model = trainer(config, train_x, train_y, num_epochs=config["epochs"], lr=config["learning_rate"], batch_size = config["batch_size"], model_type = model_type)
stats = eval_matrices(model, test_x, test_y)
stats.print()

Epoch [1/50], Step [2000/3815], Loss: 0.0369
Epoch [11/50], Step [2000/3815], Loss: 0.0003
Epoch [21/50], Step [2000/3815], Loss: 0.0004
Epoch [31/50], Step [2000/3815], Loss: 0.0003
Epoch [41/50], Step [2000/3815], Loss: 0.0000
Accuracy: 0.9966
Precision: 0.6250
Recall: 0.5344
F1 Score: 0.5761
ROC: 0.9888
PR AUC: 0.5957
Confusion Matrix
30474 	 42
61 	 70


/tmp/ipykernel_155154/1793129051.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = [torch.nn.functional.softmax(r) for r in results]


## Captum Model Interpretation

In [1]:
# !pip install captum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.5 MB/s eta 0:00:00m eta 0:00:010:00:010m


In [ ]:
{'vocab_size': 0, 'emb_size': 4, 'reshape': False, 'lstm_layers': 2, 'bi_lstm': True, 'dropout_prob': 0.4, 'batch_size': 256, 'hidden_size': 64, 'number_hidder_layers': 0, 'epochs': 60, 'learning_rate': 0.001}